# Install MySQLdb

```bash
 (tensorflow) ~$ sudo apt-get install python-dev libmysqlclient-dev
 
 (tensorflow) ~$ pip install mysqlclient

```

In [260]:
import MySQLdb
import json
import pandas as pd

In [261]:
timeData = '2006_01'

In [262]:
connection = MySQLdb.connect(host ='localhost',
                     user='root',
                     password='pwd')

In [263]:
c = connection.cursor()

# Database Creation

In [264]:
"CREATE DATABASE {};".format(timeData)

'CREATE DATABASE 2006_01;'

In [265]:
c.execute("CREATE DATABASE IF NOT EXISTS {};".format(timeData))

1

![Imgur](https://i.imgur.com/6VqnCjn.png?1)

# Database Selection

In [266]:
"USE {};".format(timeData)

'USE 2006_01;'

In [267]:
c.execute("USE {};".format(timeData))

0

# DataTable Creation

In [268]:
c.execute(
    "CREATE TABLE IF NOT EXISTS question_answer(parent_id VARCHAR(10) , comment_id VARCHAR(10), parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT, PRIMARY KEY (parent_id), UNIQUE KEY(comment_id));")

0

![Imgur](https://i.imgur.com/tW1u4ig.png?1)

In [269]:
# 데이터 하나하나 업로드하는 방식은 비효율적 따라서 아래의 방법을 적용
sql_transaction = []
# 읽어드이는 데이터량
data_counter = 0
# 실제로 추가되는 데이터량
paired_data = 0
# offset
start_data = 0
# Delete data where parent IS NULL
cleanup = 1000000

In [270]:
def transaction_bldr(sql):
    # global: 함수에 local하게 적용되는 것이 아니라 함수 밖에 있는 sql_transaction울 불러옴; preserved the values
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('START TRANSACTION;')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        # upload
        connection.commit()
        # initialize
        sql_transaction = []

# Data load from  external disk 

In [271]:
'/mnt/00B267E9B267E1A0/chatbot/{}/RC_{}'.format(timeData.split('_')[0], timeData.replace('_','-'))

'/mnt/00B267E9B267E1A0/chatbot/2006/RC_2006-01'

In [272]:
with open('/mnt/00B267E9B267E1A0/chatbot/{}/RC_{}'.format(timeData.split('_')[0], timeData.replace('_','-')), buffering=2000000000) as f:
    data = [json.loads(row) for row in f]

In [273]:
pd.DataFrame(data[0:5])

,author,author_flair_css_class,author_flair_text,body,controversiality,created_utc,distinguished,edited,gilded,id,link_id,parent_id,retrieved_on,score,stickied,subreddit,subreddit_id,ups
0,jh99,None,None,early 2006 a probable date,0,1136074029,None,False,0,c2715,t3_22569,t3_22569,1473821517,0,False,reddit.com,t5_6,0
1,jpb,None,None,If you are going to post something that has a ...,0,1136076410,None,False,0,c2717,t3_22542,t3_22542,1473821517,0,False,reddit.com,t5_6,0
2,Pichu0102,None,None,Microsoft hates it's own products?\r\nWho knew?,0,1136078623,None,False,0,c2718,t3_22515,t3_22515,1473821517,2,False,reddit.com,t5_6,2
3,libertas,None,None,"this looks interesting, but it's already aired...",0,1136079346,None,False,0,c2719,t3_22528,t3_22528,1473821517,2,False,reddit.com,t5_6,2
4,mdmurray,None,None,I have nothing but good things to say about De...,0,1136081389,None,False,0,c2722,t3_22538,t3_22538,1473821517,0,False,reddit.com,t5_6,0


In [274]:
example = data[3] 
example

{'author': 'libertas',
 'author_flair_css_class': None,
 'author_flair_text': None,
 'body': "this looks interesting, but it's already aired, and it's not like there's streaming video, so what's the point?",
 'controversiality': 0,
 'created_utc': 1136079346,
 'distinguished': None,
 'edited': False,
 'gilded': 0,
 'id': 'c2719',
 'link_id': 't3_22528',
 'parent_id': 't3_22528',
 'retrieved_on': 1473821517,
 'score': 2,
 'stickied': False,
 'subreddit': 'reddit.com',
 'subreddit_id': 't5_6',
 'ups': 2}

In [275]:
parent_id = example['parent_id']
parent_id

't3_22528'

In [276]:
body = example['body'].replace('\n', ' newlinechar ').replace('\r', ' newlinechar ').replace('"', "'")
body

"this looks interesting, but it's already aired, and it's not like there's streaming video, so what's the point?"

In [277]:
comment_id = example['id']
comment_id

'c2719'

In [278]:
created_utc = example['created_utc']
created_utc

1136079346

In [279]:
score = example['score']
score

2

In [280]:
subreddit = example['subreddit']
subreddit

'reddit.com'

# Searching previous data

- We need to get parent body correponding to comment body
- If exists parent_id in current DB then load to comment corresponding to it

In [281]:
def find_parent(pid):
    try:
        # LIMIT 1 : 출력되는 데이터들의 첫번째 행을 가져옴
        sql = "SELECT comment FROM question_answer WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        # print(str(e))
        return False

In [282]:
parent_data = find_parent(parent_id)
parent_data

False

- If exists parent_id in current DB then load to score corresponding to it

In [283]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM question_answer WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        return False

In [284]:
existing_comment_score = find_existing_score(parent_id)
existing_comment_score

False

- Body refined

In [285]:
def acceptable(data):
    # token > 1000 이상 이거나 데이터가 없을 경우
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    # 자소단위가 너무 많을 경우
    elif len(data) > 32000:
        return False
    # 삭제표시 TEXT가 들어 있는 경우
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True
    
acceptable(body)

True

- Insert new data
- sql_transaction is going to be update when more than 1000

In [286]:
def sql_insert_has_parent(commentid, parentid, parent, comment, subreddit, time, score):
    try:
        sql = """INSERT INTO question_answer (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(
            parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion', str(e))

In [293]:
"""INSERT INTO question_answer (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(
            parent_id, comment_id , parent_data, body, subreddit, created_utc, score, parent_id)

'INSERT INTO question_answer (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("t3_22528","c2719","False","this looks interesting, but it\'s already aired, and it\'s not like there\'s streaming video, so what\'s the point?","reddit.com",1136079346,2);'

- Insert new data without parent_data
- Comment body is necesssary for being parent_data

In [287]:
def sql_insert_no_parent(commentid, parentid, comment, subreddit, time, score):
    try:
        sql = """INSERT INTO question_answer (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(
            parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion', str(e))

 - If score is higher than before, update previous data

In [288]:
def sql_insert_replace_comment(commentid, parentid, parent, comment, subreddit, time, score):
    try:
        # UPDATE DATA SET parent_id => what you want to change
        sql = """UPDATE question_answer SET parent_id = "{}", comment_id = "{}", parent = "{}", comment = "{}", subreddit = "{}", unix = "{}", score = "{}" WHERE parent_id = "{}";""".format(
            parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion', str(e))

In [289]:
"""UPDATE question_answer SET parent_id = "{}", comment_id = "{}", parent = "{}", comment = "{}", subreddit = "{}", unix = "{}", score = "{}" WHERE parent_id = "{}";""".format(
            parent_id, comment_id , parent_data, body, subreddit, created_utc, score, parent_id)

'UPDATE question_answer SET parent_id = "t3_22528", comment_id = "c2719", parent = "False", comment = "this looks interesting, but it\'s already aired, and it\'s not like there\'s streaming video, so what\'s the point?", subreddit = "reddit.com", unix = "1136079346", score = "2" WHERE parent_id = "t3_22528";'

In [290]:
if existing_comment_score:
    if score > existing_comment_score:
        # acceptable: 더 유효한 데이터를 뽑아내는 작업
        if acceptable(body):
            # 데이터 교체
            sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit,
                                       created_utc, score)
# 불필요한 데이터를 전처리해서 정제시킴
else:
    # acceptable: 더 유효한 데이터를 뽑아내는 작업
    if acceptable(body):
        # if exist or not
        if parent_data:
            if score >= 2:
                sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit,
                                      created_utc, score)
                paired_data += 1
        else:
            # parent_data가 없어도 다른 정보들을 삽입
            sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)

- Print pregress
- Delete null data 

# An example for executing sql_trainsaction

In [291]:
sql_transaction

['INSERT INTO question_answer (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("t3_22528","c2719","this looks interesting, but it\'s already aired, and it\'s not like there\'s streaming video, so what\'s the point?","reddit.com",1136079346,2);']

In [292]:
c.execute('START TRANSACTION;')
for s in sql_transaction:
    try:
        c.execute(s)
    except:
        pass
# upload
connection.commit()

![Imgur](https://i.imgur.com/Kjg7wms.png?1)